# Исследование надёжности заёмщиков

# Содержание

Часть 1. Описание проекта.<br>

* 1. Описание исследования
* 2. Цель исследования

Часть 2. Предобработка данных.<br>

* 1. Изучение данных
* 2. Обработка пропусков
* 3. Замена типа данных
* 4. Обработка дубликатов

Часть 3. Дополнение данных.<br>

* 1. Лемматизация
* 2. Категоризация данных

Часть 4. Исследование зависимостей .<br>

* 1. Зависимость между наличием детей и возвратом кредита в срок 
* 2. Зависимость между семейным положением и возвратом кредита в срок
* 3. Зависимость между уровнем дохода и возвратом кредита в срок
* 4. Зависимость цели кредита на его возврат в срок

Часть 5. Общий вывод.<br>


## Часть. Описание проекта
### Описание исследования
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Цели исследования:

1. Найти зависимости между семейным положением клиента и возвратом кредита в срок
1. Найти зависимости между финансовым положением клиента и возвратом кредита в срок
2. Найти зависимость между целями клиентов и возвратом кредита.

## Часть. Предобработка данных
### Изучение данных

In [1]:
# Подключение библиотеки pandas
import pandas as pd 

In [2]:
# Чтение файла с данными и сохранение в переменную data
data = pd.read_csv('/datasets/data.csv') 
# Вывод первых 10 значений
display(data.head(10))
# Общая информация о датасете
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Строки таблицы содержат данные о заемщиках. Предварительно можно выделить следующие группы данных:
1. Общая информация о заемщике:
   * 'gender' - пол клиента
   * 'dob_years' - возраст клиента в годах
2. Семейное положение:
   * 'family_status' -  семейное положение
   * 'family_status_id' - идентификатор семейного положения
   * 'children' - количество детей в семье
3. Образование:
   * 'education' -  уровень образования клиента 
   * 'education_id' - идентификатор уровня образования
4. Трудоспособность:
   * 'income_type' - тип занятости
   * 'days_employed' - общий трудовой стаж в днях
5. Информация по кредитам:
   * 'debt' - имел ли задолженность по возврату кредитов
   * 'total_income' - ежемесячный доход
   * 'purpose' - цель получения кредита

В предоставленных данных встречаются пропуски, некоторые данные написаны в разном регистре.

### Обработка пропусков

Для поиска пропусков воспользуемся методом `'isna()'` и подсчитаем пропуски при помощи метода `'sum()'`.

In [3]:
# Поиск и подсчет пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски находятся в колонках`'days_employed'` и `'total_income'`, причем количество пропусков одинаковое, скорее всего это связано с тем, что если неизвестно количество рабочих дней, то неизвестен и ежемесячный доход. Рассчитаем долю пропусков в колонке.

In [4]:
# Процент пропусков
data.isna().mean().apply('{:.2%}'.format)

children             0.00%
days_employed       10.10%
dob_years            0.00%
education            0.00%
education_id         0.00%
family_status        0.00%
family_status_id     0.00%
gender               0.00%
income_type          0.00%
debt                 0.00%
total_income        10.10%
purpose              0.00%
dtype: object

Пропуски в колонках `'days_employed'` и `'total_income'` занимают около 10% значений в каждой. Игнорировать или удалить эти данные не выход, это может повлиять на результат исследования. При этом большая часть значений в колонке `'days_employed'` отрицательные, возможно это технический сбой (ниже приводится предположение о данных в этой колонке) возпользуемся функцией `'abs()'` для исправления.

In [5]:
# Применение abs
data['days_employed'] = abs(data['days_employed'])
# Проверка
print(data[data['days_employed'] <= 0])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


На выходе получили пустой датасет, следовательно от отрицательных значений избавились.

Рассмотрим данные в других колонках: `'children'`, `'dob_years'`, `'gender'`.

In [6]:
# Вывод значений и их количества из колонок children, dob_years, gender
print(data['children'].value_counts(),'\n')
print(data['dob_years'].value_counts(),'\n')
print(data['gender'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64 

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Заметим следующие аномалии:
1. В колонке `'children'` есть значения "-1" и "20" в количестве 47 и 76 штук соответственно.Доля этих значения составляет примерно 0,6% от всех значений в колонке. Возможно "-1" также подразумевает отсутствие детей в семье, а значение "20" связано с опечаткой при вводе данных (мб,конечно и есть люди с 20-ю детьми, но сложно представить, что среди 21 тысячи рандомных пользователей найдется аж 76 таких человек). 
2. В колонке `'dob_years'` есть 101 значение равное "0", что составляет около 0,4% от значений колонки. Возможно это тех. сбой.
3. В колонке `'gender'` есть одно значение "XNA".

Суммарно доля всех этих аномалий составляет около 1% от датасета, что вряд ли может существенно повлиять на результат исследования. Исключим строки с аномальными значениями из датасета при помощи метода `'loc()'`.

In [7]:
# Удаление аномальных значений
data = data.loc[(data['children'] > -1) & (data['children'] < 6)]
data = data.loc[data['dob_years'] != 0]
data = data.loc[data['gender'] != 'XNA']

# Проверка результата 
print(data['children'].value_counts(),'\n')
print(data['dob_years'].value_counts(),'\n')
print(data['gender'].value_counts())

0    14079
1     4802
2     2042
3      328
4       41
5        9
Name: children, dtype: int64 

35    614
41    603
40    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
44    543
29    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
55    441
59    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    262
23    252
65    194
22    183
66    183
67    167
21    110
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64 

F    14083
M     7218
Name: gender, dtype: int64


Для более точного заполнения пропусков можно связать стаж (`'days_employed'`) с возрастом (`'dob_years'`), а доход (`'total_income'`) с должностью (`'income_type'`), для этого:
1. Найдем уникальные значения для колонок `'dob_years'` и `'income_type'`, для этого воспользуемся методом `'unique()'`.


2. Запустим цикл по уникальным значениям.


3. Заменим пропуски методом `'loc'`, в колонках которые удовлетворяют двум условиям:
    * содержат пропуски;
    * соответствуют уникальному значению.
   
   Замена пропусков производится на медиальные значения при помощи метода `'median()'`. Медианы рассчитываются для каждого уникального значения.

In [8]:
# Функция замены пропусков медианным значением 
# на вход подаем список уникальных значений, имя колонки уникальных значений, имя колонки с пропусками 
def med(uniq_value, col_1, col_2):
    for uniq in uniq_value:
        data.loc[(data[col_2].isna()) & (data[col_1] == uniq), col_2] = data[data[col_1] == uniq][col_2].median()

In [9]:
# Применение функции med
med(data['income_type'].unique(), 'income_type', 'total_income')
med(data['dob_years'].unique(), 'dob_years', 'days_employed')
    
# Проверка результата    
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

На этапе обработки пропусков, найденые пропуски в  колонках `'days_employed'` и `'total_income'` были заменены на медианные значения. Также в ходе исследования данных были обнаружены аномальные значения в колонках`'children'`, `'dob_years'` и `'gender'`. Т.к. суммарно аномальные значения составляли около 1% от датасета, их влияние на исследование несущественно и ими можно пренебречь.

### Замена типа данных

Заменим тип данных в колонках `'days_employed'` и `'total_income'` с вещественного на целочисленный. Для работы с данными и выполнения исследования целочисленного типа будет хватать. Т.к. значения в колонках уже имеют тип данных float, то для перевода в int достаточно будет воспользоваться методом `'astype'`.

In [10]:
# Замена типа данных в колонках days_employed и total_income при помощи метода astype
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
# Проверка результата
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21301 non-null  int64 
 1   days_employed     21301 non-null  int64 
 2   dob_years         21301 non-null  int64 
 3   education         21301 non-null  object
 4   education_id      21301 non-null  int64 
 5   family_status     21301 non-null  object
 6   family_status_id  21301 non-null  int64 
 7   gender            21301 non-null  object
 8   income_type       21301 non-null  object
 9   debt              21301 non-null  int64 
 10  total_income      21301 non-null  int64 
 11  purpose           21301 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Предположение по поводу значений в `'days_employed'`:**

Возможно они заданы в минутах, а отрицательные значения это техническая ошибка. Вообще идея была следующая:
* рабочая неделя это 40 часов;
* в году 52 недели.

Следовательно 1 год = 2080 рабочих часов. Если в колонке заданы часы, то на примере 4ой строки, при возрасте 53 года максимальный стаж будет равен  (53 - 16) * 2080 = 76960 часов, и никак не дотягивает до 340266.

**Вывод**

Проверка результата показывает, что значения в столбцах `'days_employed'` и `'total_income'` сменились на целочисленные.

### Обработка дубликатов

Для проверки датасета на дубликаты воспользуемся методом `'value_counts'`.

In [11]:
# Поиск дубликатов в колонке education
print(data['education'].value_counts(), '\n')
# Поиск дубликатов в колонке family_status
print(data['family_status'].value_counts(), '\n')
# Поиск дубликатов в колонке income_type
print(data['income_type'].value_counts())

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      662
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64 

женат / замужем          12254
гражданский брак          4138
Не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64 

сотрудник          10996
компаньон           5033
пенсионер           3819
госслужащий         1447
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


Дубликаты обнаружены только в колонке `'education'`, причем связаны они с регистром, в других колонках дубликатов нет. Колонка `'purpose'` не рассматривалась, т.к. там  необходимо провести лемматизацию.
Чтобы избавиться от дубликатов приведем все данные к нижнему регистру при помощи метода `'lower()'`.

In [12]:
# Смена регистра методом lower
data['education'] = data['education'].str.lower()
# Проверка результата
data['education'].value_counts()

среднее                15073
высшее                  5202
неоконченное высшее      738
начальное                282
ученая степень             6
Name: education, dtype: int64

Таким образом избавились от неявных дубликатов, теперь проверим датасет на наличие явных дубликатов при помощи метода `'duplicated()'`.

In [13]:
# Проверка на явные дубликаты
data.duplicated().sum()

71

В датасете 71 явный дубликат, избавимся от них методом `'drop_duplicates()'`, а также обновим индексацию датасета методом `'reset_index()'`.

In [14]:
# Удаление дубликатов
data = data.drop_duplicates().reset_index(drop=True)
# Проверка
data.duplicated().sum()

0

**Вывод**

На данном этапе избавились от явных и неявных дубликатов в датасете.

## Часть. Дополнение данных 
### Лемматизация

Лемматизацию необходимо провести для колонки `'purpose'`. Для этого выведем уникальные значения данной колонки при помощи метода `'value_counts()'`.

In [15]:
# Вывод уникальных значений
data['purpose'].value_counts()

свадьба                                   785
на проведение свадьбы                     759
сыграть свадьбу                           755
операции с недвижимостью                  669
покупка коммерческой недвижимости         655
покупка жилья для сдачи                   647
операции с коммерческой недвижимостью     643
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
покупка своего жилья                      618
строительство жилой недвижимости          617
покупка недвижимости                      612
ремонт жилью                              602
покупка жилой недвижимости                599
на покупку своего автомобиля              501
заняться высшим образованием      

В качестве основных лемм можно выделить следующие: свадьба, недвижимость, автомобиль, образование. На основании этого предлагается добавить выделенные леммы в колонке `'simple_purpose'`.  

In [16]:
# Импрот pymystem3
from pymystem3 import Mystem
m = Mystem()

# Функция, которая возвращает лемму
def lemma(elem):
    lemmas = m.lemmatize(elem) #выполняем лемматизацию 
    if 'свадьба' in lemmas:    #здесь и далее ищем, есть ли в списке лемм нужная нам
        return 'свадьба'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'жилье' in lemmas:
        return 'недвижимость'
    elif 'недвижимость' in lemmas:
        return 'недвижимость'


In [17]:
# Применение функции lemma

# Добавляем столбец simple_purpose в датафрейм  
data['simple_purpose'] = data['purpose'].apply(lemma) 
# Проверяем результат
data['simple_purpose'].value_counts()

недвижимость    10703
автомобиль       4258
образование      3970
свадьба          2299
Name: simple_purpose, dtype: int64

**Вывод**

Выполнениа лемматизация и добавлена колонка `'simple_purpose'`. 

### Категоризация данных

На основаниии вопросов, на которые надо ответить в исследовании, следует категоризировать данные по следующим параметрам:
1. возрасту (создание классификации по возрасту)
2. наличию детей (создание классификации по количеству детей)
3. уровню дохода (создание классификации по доходу) 
4. семейному статусу (создание словаря, включающего статус семьи и id статуса)
5. образованию (создание словаря, включающего статус образования и id статуса).

In [18]:
# Категоризация по типу для семейного статуса, создание словаря
family_dict = dict(zip(data['family_status_id'], data['family_status']))
display(family_dict)

# Категоризация по типу для образования, создание словаря
education_dict = dict(zip(data['education_id'], data['education']))
display(education_dict)

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'Не женат / не замужем'}

{0: 'высшее',
 1: 'среднее',
 2: 'неоконченное высшее',
 3: 'начальное',
 4: 'ученая степень'}

In [19]:
# Категоризация по возрасту
data['dob_years'].value_counts()

def age(elem):
    if elem < 44:
        return 'молодой'
    if 45 <= elem < 60:
        return 'средний'
    if 60 <= elem:
        return 'старый'
    
# Применение функции age
data['age_group'] = data['dob_years'].apply(age)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,simple_purpose,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,молодой
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,молодой
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,молодой
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,молодой
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний


In [20]:
# Категоризация по доходу
data['total_income'].value_counts()

def income(elem):
    if elem < 100000:
        return 'низкий'
    if 100000 <= elem < 200000:
        return 'средний'
    if 200000 <= elem:
        return 'высокий'
    
# Применение функции income
data['income_group'] = data['total_income'].apply(income)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,simple_purpose,age_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,молодой,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,молодой,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,молодой,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,молодой,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний,средний


In [21]:
# Категоризация по количеству детей
data['children'].value_counts()

def child(elem):
    if elem == 0:
        return 'бездетные'
    if 1 <= elem <= 2:
        return 'малодетные'
    if 2 < elem:
        return 'многодетные'

# Применение функции child
data['child_group'] = data['children'].apply(child)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,simple_purpose,age_group,income_group,child_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,молодой,высокий,малодетные
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,молодой,средний,малодетные
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,молодой,средний,бездетные
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,молодой,высокий,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний,средний,бездетные


**Вывод**

Выполнена категоризация данных.

## Часть. Исследование зависимостей
### Зависимость между наличием детей и возвратом кредита в срок 

In [22]:
# Создаем сводную таблицу
child_data = data.pivot_table(index='child_group', columns='debt', values='children', aggfunc='count')
# Добавляем колонку с отношением должников к общему количеству кредиторов в группе (в процентах)
child_data['ratio'] = child_data[1] / (child_data[0] + child_data[1]) * 100
# Выводим сводную таблицу
display(child_data.sort_values(by='ratio', ascending = False))

debt,0,1,ratio
child_group,,,
малодетные,6196,635,9.295857
многодетные,347,31,8.201058
бездетные,12963,1058,7.545824


В малодетных семьях наибольший процент должников, далее следуют многодетные семьи, при этом наименьший процент должнико в бездетных семьях.

Посмотрим, как будут выглядеть таблица, если дополнительно разделить ее на категории, связанные с доходом из колонки `'income_group'`. 

In [23]:
# Создаем сводную таблицу
child_data = data.pivot_table(index=['child_group', 'income_group'], columns='debt', values='children', aggfunc='count')
# Добавляем колонку с отношением должников к общему количеству кредиторов в группе (в процентах)
child_data['ratio'] = child_data[1] / (child_data[0] + child_data[1]) * 100
# Выводим сводную таблицу
display(child_data.sort_values(by='ratio', ascending = False))

debt                         0    1      ratio
child_group income_group                      
многодетные низкий          57    9  13.636364
малодетные  средний       3453  381   9.937402
            низкий        1200  124   9.365559
многодетные средний        195   17   8.018868
бездетные   средний       7133  618   7.973165
малодетные  высокий       1543  130   7.770472
бездетные   низкий        2811  220   7.258331
            высокий       3019  220   6.792220
многодетные высокий         95    5   5.000000

**Вывод**

Можно увидеть, следующее:
* Основа малодетных семей - семьи со средним и низким доходом среди которых ~9% должников.
* Наибольший процент по задолженности в многодетных семьях с низким доходом, далее идут малодетные семьи со средним и низким доходом.  В обеспеченных многодетных семьях наоборот самый низкий процент задолженностей. При этом эти два факта компенсируют друг друга, из-за чего при общем рассмотрении зависимости долга по кредиту и наличию детей многодетные семьи на 2ом месте. 
* Количество должников в бездетных семьях особо не зависит от дохода, колебания составляют ~1%.  

### Зависимость между семейным положением и возвратом кредита в срок

In [24]:
# Создаем сводную таблицу
family_data = data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
# Добавляем колонку с отношением должников к общему количеству кредиторов в группе (в процентах)
family_data['ratio'] = family_data[1] / (family_data[0] + family_data[1]) * 100
# Выводим сводную таблицу
display(family_data.sort_values(by='ratio', ascending = False))

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2508,272,9.784173
гражданский брак,3729,383,9.314202
женат / замужем,11290,923,7.557521
в разводе,1095,84,7.124682
вдовец / вдова,884,62,6.553911


Наибольшее число должников не состоят в браке или состоят в гражданском браке, далее следуют те, кто имеет семью или в разводе, наименьшее число должников среди вдовцов/вдов.

Посмотрим, как будут выглядеть таблица, если дополнительно разделить ее на категории, связанные с доходом из колонки `'age_group'`. 

In [25]:
# Создаем сводную таблицу
family_data = data.pivot_table(index=['family_status', 'age_group'], columns='debt', values='family_status_id', aggfunc='count')
# Добавляем колонку с отношением должников к общему количеству кредиторов в группе (в процентах)
family_data['ratio'] = family_data[1] / (family_data[0] + family_data[1]) * 100
# Выводим сводную таблицу
display(family_data.sort_values(by='ratio', ascending = False))

debt                                0    1      ratio
family_status         age_group                      
Не женат / не замужем молодой    1627  208  11.335150
гражданский брак      молодой    2059  248  10.749892
женат / замужем       молодой    5777  578   9.095201
гражданский брак      средний    1183  106   8.223429
Не женат / не замужем средний     603   52   7.938931
в разводе             молодой     470   40   7.843137
                      старый      144   11   7.096774
вдовец / вдова        старый      371   27   6.783920
                      средний     421   29   6.444444
женат / замужем       средний    3966  264   6.241135
в разводе             средний     452   30   6.224066
вдовец / вдова        молодой      80    5   5.882353
гражданский брак      старый      375   18   4.580153
женат / замужем       старый     1248   58   4.441041
Не женат / не замужем старый      230    9   3.765690

**Вывод**

* Молодые должники, тянут вверх показатели задолженностей для групп "Не женат / не замужем" и "гражданский брак". Также молодые люди составляют основу этих групп.  
* Для тех, кто состоит в браке наоборот, возрастные должники (средние и старые) тянут показатели задолженности вниз, т.к. количество молодых и возрастных должников сопоставимо.
* Количество должников в разводе особо не зависит от возраста, колебания составляют ~1%.  
* Наименьшее количество должников среди вдовцов / вдов.  

### Зависимость между уровнем дохода и возвратом кредита в срок

In [26]:
# Создаем сводную таблицу
income_data = data.pivot_table(index='income_group', columns='debt', values='total_income', aggfunc='count')
# Добавляем колонку с отношением должников к общему количеству кредиторов в группе (в процентах)
income_data['ratio'] = income_data[1] / (income_data[0] + income_data[1]) * 100
# Выводим сводную таблицу
display(income_data.sort_values(by='ratio', ascending = False))

debt,0,1,ratio
income_group,,,
средний,10781,1016,8.612359
низкий,4068,353,7.984619
высокий,4657,355,7.083001


**Вывод**

Наибольшее число должников со средним доходом.

### Зависимость цели кредита на его возврат в срок

In [27]:
# Создаем сводную таблицу
purpose_data = data.pivot_table(index='simple_purpose', columns='debt', values='purpose', aggfunc='count')
# Добавляем колонку с отношением должников к общему количеству кредиторов в группе (в процентах)
purpose_data['ratio'] = purpose_data[1] / (purpose_data[0] + purpose_data[1])  * 100
# Выводим сводную таблицу
display(purpose_data.sort_values(by='ratio', ascending = False))

debt,0,1,ratio
simple_purpose,,,
автомобиль,3861,397,9.323626
образование,3601,369,9.294710
свадьба,2118,181,7.872988
недвижимость,9926,777,7.259647


Наибольшее число должников среди тех, кто берет кредит на авто или образование.

Рассмотрим, как будут выглядеть таблица, если дополнительно разделить ее на категории, связанные с доходом из колонки `'income_group'`. 

In [28]:
# Создаем сводную таблицу
purpose_data = data.pivot_table(index=['simple_purpose', 'income_group'], columns='debt', values='purpose', aggfunc='count')
# Добавляем колонку с отношением должников к общему количеству кредиторов в группе (в процентах)
purpose_data['ratio'] = purpose_data[1] / (purpose_data[0] + purpose_data[1])  * 100
# Выводим сводную таблицу
display(purpose_data.sort_values(by='ratio', ascending = False))

debt                            0    1      ratio
simple_purpose income_group                      
автомобиль     средний       2141  240  10.079798
образование    низкий         754   81   9.700599
               средний       2030  218   9.697509
свадьба        низкий         438   44   9.128631
               средний       1164  109   8.562451
автомобиль     низкий         784   72   8.411215
               высокий        936   85   8.325171
образование    высокий        817   70   7.891770
недвижимость   средний       5446  449   7.616624
               низкий        2092  156   6.939502
               высокий       2388  172   6.718750
свадьба        высокий        516   28   5.147059

**Вывод**

* Должники со средним доходом тянут вверх общую долю должников, взявших кредит на автомобиль.
* В случае с образованием должники с высоким доходом снижают общую долю должников.
* Должники с высоким доходом снижают общую долю должников, взявших кредит на свадьбу.
* Количество должников взявших кредит на недвижимость особо не зависит от дохода, колебания составляют ~1%.

## Часть. Общий вывод

В ходе исследования проверены следующие зависимости:
1. Зависимость между наличием детей и возвратом кредита в срок.

В рамках данной проверке выяснилось, что наибольшее число должников среди семей, в которых 1-2 ребенка, наименьшее в бездетных семьях.

2. Зависимость между семейным положением и возвратом кредита в срок.

Больше всего должников среди тех, кто не состоит в браке или состоит в гражданском браке, наименьшее у вдовцов/вдов.

3. Зависимость между уровнем дохода и возвратом кредита в срок.

Наибольшее число должников со средним доходом.

4. Влияние цели кредита на его возврат в срок.

Наибольшее число должников среди тех, кто берет кредит на авто или образование.